# Forward model for M/EEG source analysis

This notebook illustrates how to use features of the `stormdb.process`-module to:

* generate a BEM model based on pre-calculated tissue surfaces (_e.g._, inner skull, outer skull and outer skin)
* solve the BEM model (calculate interpolation coefficients for the surface elements based on node values)
* generate a _source space_ of possible dipole locations for a distributed (_e.g._, MNE) estimate
    * based on the cortical sheet (white-matter boundary) extracted using _Freesurfer_ `recon-all`
* generate a _forward operator_ for source modeling, using
    * solved BEM model
    * source space
    * head-to-mri transformation
    * an MEG measurement file containing the locations of the sensors relative to the head

_All computations are performed on the `hyades`-cluster._ The underlying code is based on the implementations found in `mne-python` and all outputs are written in the Neuromag FIF-format.

## Step 1: BEM creation and solution



### Specify conductivity parameters

In [1]:
import os.path as op
from stormdb.process import MNEPython
from stormdb.base import mkdir_p  # mkdir -p

In [2]:
proj_name = 'MINDLAB2015_MEG-Sememene'
subject = '0010_HXN'  # full code must be given!
subjects_dir = op.join('/projects', proj_name, 'scratch', 'fs_subjects_dir')
fwd_dir = op.join('/projects', proj_name, 'scratch', 'fwd_operators')
mkdir_p(fwd_dir)  # this must exist

In [3]:
mp = MNEPython(proj_name)

In [4]:
mp.prepare_bem_model(subject, op.join(subjects_dir, subject, 'bem', 'test-3L-sol.fif'),
                     subjects_dir=subjects_dir, conductivity=[0.3, 0.006, 0.3])

In [7]:
mp.submit(fake=True)

Following command would be submitted (if not fake)
python -c "from mne import make_bem_model, make_bem_solution, write_bem_solution;surfs = make_bem_model('0010_HXN', ico=None, conductivity=[0.3, 0.006, 0.3], subjects_dir='/projects/MINDLAB2015_MEG-Sememene/scratch/fs_subjects_dir');bem = make_bem_solution(surfs);write_bem_solution('/projects/MINDLAB2015_MEG-Sememene/scratch/fs_subjects_dir/0010_HXN/bem/test-3L-sol.fif', bem)"
Following command would be submitted (if not fake)
python -c "from mne import setup_source_space, write_source_spaces;src = setup_source_space('0010_HXN');write_source_spaces(src, '/projects/MINDLAB2015_MEG-Sememene/scratch/fs_subjects_dir/0010_HXN/bem/test-ico6-src.fif')"


In [6]:
src_fname = op.join(subjects_dir, subject, 'bem', 'test-ico6-src.fif')
mp.setup_source_space(subject, src_fname)

In [8]:
mp.submit()

Cluster job submitted, job ID: 3829439
Cluster job submitted, job ID: 3829440


In [10]:
mp.status

#1 (3829439): Running on hyades04 (short.q)
#2 (3829440): Running on hyades06 (short.q)
